In [7]:
import json
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *
from config import snowflake_conn_prop
import pandas as pd

session = Session.builder.configs(snowflake_conn_prop).create()

In [8]:
# how I would do in pandas
df = pd.read_csv('./medium_data.csv', index_col=0)
df.head(5)
# 6508 urls

,url,title,subtitle,image,claps,responses,reading_time,publication,date
id,,,,,,,,,
1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,NaN,1.png,850,8,8,Towards Data Science,2019-05-30
2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,NaN,2.png,1100,11,9,Towards Data Science,2019-05-30
3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,1,5,Towards Data Science,2019-05-30
4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,0,4,Towards Data Science,2019-05-30
5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,3,4,Towards Data Science,2019-05-30


In [9]:
session.file.put('./medium_data.csv', '@~/', overwrite=True)
schema = StructType([
  StructField("id", IntegerType()),
  StructField("url", StringType()),
  StructField("title", StringType()),
  StructField("subtitle", StringType()),
  StructField("image", StringType()),
  StructField("claps", IntegerType()),
  StructField("responses", VariantType()),
  StructField("reading_time", IntegerType()),
  StructField("publication", StringType()),
  StructField("date", DateType())
])
df = session.read.option("skip_header", 1).option("field_delimiter", ",").option("FIELD_OPTIONALLY_ENCLOSED_BY", "\"").schema(schema).csv('@~/medium_data.csv')
df.to_pandas().set_index('ID').head(5)

,URL,TITLE,SUBTITLE,IMAGE,CLAPS,RESPONSES,READING_TIME,PUBLICATION,DATE
ID,,,,,,,,,
1,https://towardsdatascience.com/a-beginners-gui...,A Beginner’s Guide to Word Embedding with Gens...,None,1.png,850,"""8""",8,Towards Data Science,2019-05-30
2,https://towardsdatascience.com/hands-on-graph-...,Hands-on Graph Neural Networks with PyTorch & ...,None,2.png,1100,"""11""",9,Towards Data Science,2019-05-30
3,https://towardsdatascience.com/how-to-use-ggpl...,How to Use ggplot2 in Python,A Grammar of Graphics for Python,3.png,767,"""1""",5,Towards Data Science,2019-05-30
4,https://towardsdatascience.com/databricks-how-...,Databricks: How to Save Files in CSV on Your L...,When I work on Python projects dealing…,4.jpeg,354,"""0""",4,Towards Data Science,2019-05-30
5,https://towardsdatascience.com/a-step-by-step-...,A Step-by-Step Implementation of Gradient Desc...,One example of building neural…,5.jpeg,211,"""3""",4,Towards Data Science,2019-05-30


In [10]:
df_tldr = df.limit(1).select(
    *df.columns, call_builtin('url_resolve_and_summarize', df.url).alias("TL_DR")
    )
df_tldr.show()

Failed to execute query [queryID: 01a48e32-0604-34e4-0000-0971243ef7b6]  SELECT  *  FROM ( SELECT "ID", "URL", "TITLE", "SUBTITLE", "IMAGE", "CLAPS", "RESPONSES", "READING_TIME", "PUBLICATION", "DATE", url_resolve_and_summarize("URL") AS "TL_DR" FROM ( SELECT  *  FROM ( SELECT $1::INT AS "ID", $2::STRING AS "URL", $3::STRING AS "TITLE", $4::STRING AS "SUBTITLE", $5::STRING AS "IMAGE", $6::INT AS "CLAPS", $7::VARIANT AS "RESPONSES", $8::INT AS "READING_TIME", $9::STRING AS "PUBLICATION", $10::DATE AS "DATE" FROM @~/medium_data.csv( FILE_FORMAT  => '"JEFFHOLLAN_DEMO"."PUBLIC".SNOWPARK_TEMP_FILE_FORMAT_WQUJOI0Y4X')) LIMIT 1)) LIMIT 10
100351 (P0000): Request failed for external function URL_RESOLVE_AND_SUMMARIZE with remote service error: 400 '{
"data":[
[0,"https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92"]
]
}'; requests batch-id: 01a48e32-0604-34e4-0000-0971243ef7b6:1:4:0:0; request batch size: 1 rows; request retries: 0; respon

SnowparkSQLException: (1304): 100351 (P0000): Request failed for external function URL_RESOLVE_AND_SUMMARIZE with remote service error: 400 '{
"data":[
[0,"https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92"]
]
}'; requests batch-id: 01a48e32-0604-34e4-0000-0971243ef7b6:1:4:0:0; request batch size: 1 rows; request retries: 0; response time (last retry): 1.49883s